In [1]:
out_dir = "025_vs_026"

In [2]:
!g++ ../../../KKT89/Combine.cpp -O3 -Ofast -march=native -std=c++17

In file included from ../../../KKT89/src/Goose.hpp:3,
                 from ../../../KKT89/src/Stage.hpp:4,
                 from ../../../KKT89/src/Duct.hpp:7,
                 from ../../../KKT89/src/Duct.cpp:2,
                 from ../../../KKT89/Combine.cpp:3:
../../../KKT89/src/Evaluator2.hpp: In member function ‘void hungry_geese::evaluation_function::BitBoard::ShiftRight()’:
../../../KKT89/src/library.hpp:64:9: warning: format ‘%d’ expects argument of type ‘int’, but argument 3 has type ‘hungry_geese::evaluation_function::BitBoard::ull’ {aka ‘long long unsigned int’} [-Wformat=]
   64 |         "`%s` (%d) is out of range [%d, %d)", #value, value, left, right)
      |         ^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
../../../KKT89/src/library.hpp:54:24: note: in definition of macro ‘ASSERT’
   54 |                 printf(__VA_ARGS__); \
      |                        ^~~~~~~~~~~
../../../KKT89/src/Evaluator2.hpp:582:9: note: in expansion of macro ‘ASSERT_RANGE’
  582 |         ASSER

In file included from ../../../KKT89/src/Duct.hpp:10,
                 from ../../../KKT89/src/Duct.cpp:2,
                 from ../../../KKT89/Combine.cpp:3:
../../../KKT89/src/Evaluator2.hpp: In member function ‘void hungry_geese::evaluation_function::nn::Model<in_dim, out_dim, hidden_1, hidden_2>::LoadParameters(const char*) [with int in_dim = 2458; int out_dim = 5; int hidden_1 = 256; int hidden_2 = 32]’:
../../../KKT89/src/Evaluator2.hpp:409:14: warning: ignoring return value of ‘size_t fread(void*, size_t, size_t, FILE*)’, declared with attribute warn_unused_result [-Wunused-result]
  409 |         fread(&embed.parameters.weight, embed.parameters.weight.Ravel().size(), sizeof(short), fp);
      |         ~~~~~^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
../../../KKT89/src/Evaluator2.hpp:410:14: warning: ignoring return value of ‘size_t fread(void*, size_t, size_t, FILE*)’, declared with attribute warn_unused_result [-Wunused-result]
  410 

In [3]:
!ls ../../rl/parameters

021_01.bin  023_01.bin	025_02.bin  026_01.bin


In [4]:
!mkdir {out_dir}
!mkdir {out_dir}/1vs3
!mkdir {out_dir}/2vs2
!mkdir {out_dir}/3vs1

In [5]:
# コマンド確認
param1 = "../../rl/parameters/025_02.bin"
param2 = "../../rl/parameters/026_01.bin"
!./a.out -j -o ./ -t 0.1 -p {param1} {param2} {param1} {param2}

In [6]:
from subprocess import Popen
N_MATCHES = 50000
p1 = Popen(f"seq 1 {N_MATCHES} | xargs -I@ -P 20 sh -c './a.out -j -o {out_dir}/1vs3/ -t 0.1 -p {param1} {param2} {param2} {param2}'", shell=True)
p2 = Popen(f"seq 1 {N_MATCHES} | xargs -I@ -P 20 sh -c './a.out -j -o {out_dir}/2vs2/ -t 0.1 -p {param1} {param1} {param2} {param2}'", shell=True)
p3 = Popen(f"seq 1 {N_MATCHES} | xargs -I@ -P 20 sh -c './a.out -j -o {out_dir}/3vs1/ -t 0.1 -p {param1} {param1} {param1} {param2}'", shell=True)

In [2]:
!ls -1 {out_dir}/1vs3 | wc -l
!ls -1 {out_dir}/2vs2 | wc -l
!ls -1 {out_dir}/3vs1 | wc -l

11067
11158
11022


In [3]:
import numpy as np

print("古いほう")
ranks_param1 = []
files = !ls -1 {out_dir}/3vs1
for file in files:
    ranks = !tail {out_dir}/3vs1/{file} -n 1
    ranks = list(map(int, ranks[0].split()))
    ranks_param1.append(ranks[0])
    ranks_param1.append(ranks[1])
    ranks_param1.append(ranks[2])
rank_count = np.bincount(ranks_param1)[1:]
print("3vs1", rank_count, rank_count / rank_count.sum())

ranks_param1 = []
files = !ls -1 {out_dir}/2vs2
for file in files:
    ranks = !tail {out_dir}/2vs2/{file} -n 1
    ranks = list(map(int, ranks[0].split()))
    ranks_param1.append(ranks[0])
    ranks_param1.append(ranks[1])
rank_count = np.bincount(ranks_param1)[1:]
print("2vs2", rank_count, rank_count / rank_count.sum())

ranks_param1 = []
files = !ls -1 {out_dir}/1vs3
for file in files:
    ranks = !tail {out_dir}/1vs3/{file} -n 1
    ranks = list(map(int, ranks[0].split()))
    ranks_param1.append(ranks[0])
rank_count = np.bincount(ranks_param1)[1:]
print("1vs3", rank_count, rank_count / rank_count.sum())


print("新しいほう")
ranks_param2 = []
files = !ls -1 {out_dir}/3vs1
for file in files:
    ranks = !tail {out_dir}/3vs1/{file} -n 1
    ranks = list(map(int, ranks[0].split()))
    ranks_param2.append(ranks[-1])
rank_count = np.bincount(ranks_param2)[1:]
print("3vs1", rank_count, rank_count / rank_count.sum())

ranks_param2 = []
files = !ls -1 {out_dir}/2vs2
for file in files:
    ranks = !tail {out_dir}/2vs2/{file} -n 1
    ranks = list(map(int, ranks[0].split()))
    ranks_param2.append(ranks[2])
    ranks_param2.append(ranks[3])
rank_count = np.bincount(ranks_param2)[1:]
print("2vs2", rank_count, rank_count / rank_count.sum())

ranks_param2 = []
files = !ls -1 {out_dir}/1vs3
for file in files:
    ranks = !tail {out_dir}/1vs3/{file} -n 1
    ranks = list(map(int, ranks[0].split()))
    ranks_param2.append(ranks[1])
    ranks_param2.append(ranks[2])
    ranks_param2.append(ranks[3])
rank_count = np.bincount(ranks_param2)[1:]
print("1vs3", rank_count, rank_count / rank_count.sum())

古いほう
3vs1 [9453 7655 7813 8145] [0.28588278 0.23150668 0.23628501 0.24632553]
2vs2 [5666 4952 5489 6209] [0.25389855 0.22190357 0.24596702 0.27823087]
1vs3 [2145 2072 3052 3798] [0.19381946 0.18722328 0.27577483 0.34318243]
新しいほう
3vs1 [2872 2394 3289 2467] [0.26056977 0.21720196 0.29840319 0.22382508]
2vs2 [6823 5168 5849 4476] [0.30574476 0.23158272 0.26209894 0.20057358]
1vs3 [10267  7967  8116  6851] [0.30923767 0.23996265 0.24445047 0.20634921]


In [4]:
!zip -r -q {out_dir}.zip {out_dir}